### Basic imports for the project

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from moviepy.editor import ImageSequenceClip

from stable_baselines3.common.callbacks import BaseCallback

### Imports that will emulate the game and an action space

In [ ]:
import gym
import retro

### Imports for our PPO algorithm (Proximal Policy Optimization) and for pre-processing (grayscale and frame stacking)

In [ ]:
from stable_baselines3 import PPO

from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

### Using GPU for training (with AMD's ROCm), checking if we have access to Cuda (at least 4 times faster training)

In [ ]:
import torch
torch.cuda.is_available()

### Hijacking the default action space with a wrapper that will allow us to
### create our own combos of buttons

In [ ]:
combos = [
    [],
    ["RIGHT"],
    ["RIGHT","Y"],
    ["RIGHT","A"],
    ["RIGHT","Y","A"],
    ["RIGHT","Y","B"],
    ["LEFT","Y"],
    ["LEFT","Y","A"],
    ["LEFT","Y","B"],
    ["A"],
    ["B"]
    ]

In [ ]:
class Discretizer(gym.ActionWrapper):

    def __init__(self, env, combos):
        super().__init__(env)
        assert isinstance(env.action_space, gym.spaces.MultiBinary)
        buttons = env.unwrapped.buttons
        self._decode_discrete_action = []
        for combo in combos:
            arr = np.array([False]*env.action_space.n)
            for button in combo:
                arr[buttons.index(button)] = True
            self._decode_discrete_action.append(arr)

        self.action_space = gym.spaces.Discrete(len(self._decode_discrete_action))

    def action(self, act):
        return self._decode_discrete_action[act].copy()


class MaRLioDiscretizer(Discretizer):

    def __init__(self, env):
        super().__init__(env=env, combos=combos)

### Storing the path of our custom data.json and scenario.json files into variables

In [ ]:
data_path = "./data.json"
scenario_path = "./scenario.json"

### data.json will have memory addresses for things that we want to keep track of
### and will be accessed from the info variable once we run the environment
### scenario.json will be our way of calculating MaRLio's reward
### Let's see how they look like. 

### data.json

````
{
  "info": {
    "checkpoint": {
      "address": 5070,
      "type": "|i1"
    },
    "coins": {
      "address": 8261055,
      "type": "|u1"
    },
    "end_of_level": {
      "address": 8259846,
      "type": "|i1"
    },
    "lives": {
      "address": 8261054,
      "type": "|i1"
    },
    "powerups": {
      "address": 25,
      "type": "|i1"
    },
    "score": {
      "address": 8261428,
      "type": "<u4"
    },
    "xpos": {
      "address": 148,
      "type": "<u2"
    },
    "ypos": {
      "address": 114,
      "type": "<u4"
    },
    "yoshi_coins": {
      "address": 8262690,
      "type": "<u4"
    }
  }
}
````

### scenario.json

````
{
  "crop": [
    0,
    0,
    0,
    0
  ],
  "done": {
    "variables": {
      "lives": {
        "op": "equal",
        "reference": -1
      }
    }
  },
  "reward": {
    "variables": {
      "score": {
        "reward": 1.0
      },
      "coins": {
        "reward": 1.0
      },
      "yoshi_coins": {
        "reward": 1.0
      },
      "xpos": {
        "op": "greater",
        "reference": "last",
        "reward": 10
      },
      "ypos": {
        "op": "less",
        "reference": "last",
        "reward": 10
      },
      "xpos": {
        "op": "less",
        "reference": "last",
        "penalty": 0.1
      },
      "lives": {
        "op": "less",
        "reference": "last",
        "penalty": 1000.0
      },
      "end_of_level": {
        "op": "equal",
        "reference": 1,
        "reward": 1000000.0
      }
    }
  }
}
````

### Let's load our game, Super Mario World (SMW) from the Super Nintendo Entertainment System (SNES)
### using scenario and info kwargs, we can load our custom data.json and scenario.json
### using record kwarg, we can save the training sessions for future viewing

In [ ]:
# saves recording to ./recordings/train/

env = retro.make(game="SuperMarioWorld-Snes", state="YoshiIsland1", record="./recordings/train/", scenario=scenario_path, info=data_path)

In [ ]:
# saves recording to ./recordings/pred/

# env = retro.make(game="SuperMarioWorld-Snes", state="YoshiIsland1", record="./recordings/pred/", scenario=scenario_path, info=data_path)

In [ ]:
# for video only, no recording

# env = retro.make(game="SuperMarioWorld-Snes", state="YoshiIsland1", scenario=scenario_path, info=data_path)

### This is where we wrap the environment in our own action space

In [ ]:
disc_env = MaRLioDiscretizer(env)

In [ ]:
obs = disc_env.reset()

### Let's check the index it returns and what combo we get from it

In [ ]:
action = disc_env.action_space.sample()
print(action)
combos[action]

### Now we can use that action inside our environment, one frame at a time
### and we can use tuple unpacking, to get access to some important information about our environment

In [ ]:
obs, reward, terminated, info = disc_env.step(action)
print(f"score: {reward}\nterminated: {terminated}\ninfo: {info}")

### The image we get is a 3-dimensional tensor with a height of 224 pixels, a width of 256 pixels, and 3 color channels (RGB)

In [ ]:
obs.shape

### Let's visualize that frame

In [ ]:
plt.matshow(obs)

### Let's look at Mario doing random actions

In [ ]:
# state = disc_env.reset()

# done = False
# for _ in range(1000):
#     action = disc_env.action_space.sample()
#     state, reward, done, info = disc_env.step(action)
#     disc_env.render()
#     time.sleep(0.005)
# disc_env.render(close=True)

### Let's pre-process our images
### First, grayscale. that'll reduce the size of our environment by 66%
### Second, let's vectorize everything
### Third, let's stack 4 frames, our model will get a sense of momentum
### Normalize the pixels, between 0-1 instead of 0-255

In [ ]:
class NormalizeObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(NormalizeObservation, self).__init__(env)
    
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0

In [ ]:
disc_env = GrayScaleObservation(disc_env, keep_dim=True)
disc_env = DummyVecEnv([lambda:disc_env])
disc_env = VecFrameStack(disc_env, 4, channels_order="last")
disc_env = NormalizeObservation(disc_env)

In [ ]:
state = disc_env.reset()

### Let's look at a frame (grayscale)

In [ ]:
plt.matshow(state[0])

### This time, the (1, 224, 256, 4) shape indicates that the observation is a stack of 4 frames, 
### where each frame is a "color" (grayscale, 1 RGB channel) image with a resolution of 224x256 pixels.

In [ ]:
state.shape

### Taking 4 steps, so we don't have blank frames

In [ ]:
for _ in range(3):
    state, reward, done, info = disc_env.step([disc_env.action_space.sample()])

### Let's visualize our stack of frames, with random actions from our list of combos
### We should see some movement

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

In [ ]:
state = disc_env.reset()

### Defining a class that will create our logs and save our model at every check_freq

In [ ]:
# callback helper function

# logs and model saving
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, "best_model_{}".format(self.n_calls))
            self.model.save(model_path)

        return True

### Creating variables that points to our paths for saving the model and the logs (tensorboard)

In [ ]:
CHECKPOINT_DIR = "./train/"
LOG_DIR = "./logs/"

#### Proximal Policy Optimization (PPO) is a reinforcement learning algorithm that is used to optimize policies for continuous control problems.  
#### The PPO algorithm uses a surrogate objective function to update the policy parameters in an iterative manner.  

#### The formula for the PPO objective function is as follows:

    LCLIP(θ) = E[min(r(θ)A, clip(r(θ), 1 - ε, 1 + ε)A)]

#### Where:

    θ represents the policy parameters
    r(θ) represents the ratio of the probabilities of taking the action under the new policy and the old policy
    A represents the advantage function, which estimates how much better a particular action is compared to the average action
    ε is a hyperparameter that determines the degree of clipping that is applied to the ratio r(θ)

#### The PPO algorithm maximizes this objective function by updating the policy parameters using stochastic gradient ascent.  
#### In addition, the algorithm includes an entropy bonus term to encourage exploration and prevent the policy from becoming too deterministic.

![Alt text](ppo.png)

### Keeping track of some hyperparameters for finetuning

In [ ]:
check_freq = 100_000
total_timesteps = 1_000_000

learning_rate = lambda f:f*0.000001
# learning_rate = 0.000001
n_steps = 256
clip_range = 0.2
ent_coef = 0.0

### Instance of our PPO algorithm, with the rest of the hyperparameters set to default

In [ ]:
callback = TrainAndLoggingCallback(check_freq=check_freq, save_path=CHECKPOINT_DIR)

In [ ]:
model = PPO(
    "CnnPolicy", 
    disc_env, 
    learning_rate=learning_rate, 
    n_steps=n_steps, 
    batch_size=64, 
    n_epochs=10, 
    gamma=0.99, 
    gae_lambda=0.95, 
    clip_range=clip_range, 
    clip_range_vf=None, 
    normalize_advantage=True, 
    ent_coef=ent_coef, 
    vf_coef=0.5, 
    max_grad_norm=0.5, 
    use_sde=False, 
    sde_sample_freq=-1, 
    target_kl=None,  
    tensorboard_log=LOG_DIR, 
    policy_kwargs=None, 
    verbose=0, 
    seed=42, 
    device='auto', 
    _init_setup_model=True
    )

### This is where MaRLio starts his adventure (training)

In [ ]:
model.learn(
    total_timesteps=total_timesteps, 
    progress_bar=True, 
    tb_log_name="ppo",
    callback=[
    callback
    ]
)

### Let's add a powerup dictionnary, we will see what powerups MaRLio has

In [ ]:
powerup_dict = {
    0: "small",
    1: "big",
    2: "cape",
    3: "fire"
}

### Let's add a dictionnary of our overlays

In [ ]:
combo_images = {
    0: "./controller/blank.png",
    1: "./controller/right.png",
    2: "./controller/righty.png",
    3: "./controller/righta.png",
    4: "./controller/rightya.png",
    5: "./controller/rightyb.png",
    6: "./controller/lefty.png",
    7: "./controller/leftya.png",
    8: "./controller/leftyb.png",
    9: "./controller/a.png",
    10: "./controller/b.png"
}

### Let's load the models and predict the environment with the models' knowledge

In [ ]:
# game loop for predict

def pred(model):
    total_reward = 0
    action_dict = {}
    powerup = ""
    check = False
    frame = 1

    state = disc_env.reset()
    done = False
    while not done:
        action, _ = model.predict(state)

        action_dict[frame] = action[0]
        state, reward, done, info = disc_env.step(action)

        if info[0]["powerups"] != powerup:
            powerup = info[0]["powerups"]
            print(powerup_dict.get(powerup, "Invalid integer value"))

        if info[0]["checkpoint"] == 1 and not check:
            check = True
            print("Midlevel Checkpoint Reached: +500k reward")

        if info[0]["end_of_level"] == 1:
            print("End of level Reached: +1m reward")

        frame += 1

        total_reward += reward
        disc_env.render()
        time.sleep(0.005)

    print("Total reward:", total_reward[0])
    print()
    return action_dict

### Helper function that will convert our .bk2 to .mp4

In [ ]:
def bk2_to_mp4():
    bk2_files = [f for f in os.listdir('./recordings/pred/') if f.endswith('.bk2')]
    bk2_files.sort()
    for bk2_file in bk2_files:
        print(f"converting {bk2_file} into .mp4")
        command = f"python3.8 -m retro.scripts.playback_movie ./recordings/pred/videos/{bk2_file}"
        os.system(command)

### Helper function that will create an overlay video

In [ ]:
def generate_video(combo_images, action_dict, filename, fps):
    print(f"Generating a video of the overlay for {filename}")
    images = []
    for _, action_index in action_dict.items():
        image_path = combo_images[action_index]
        with Image.open(image_path) as image:
            images.append(np.array(image))
    clip = ImageSequenceClip(images, fps=fps)
    clip = clip.set_duration(len(action_dict) / fps)
    clip = clip.set_position(('center', 'center'))
    clip.write_videofile(filename, fps=fps, codec='mpeg4')

### Our main function that will call our helper functions in turn, automating the process of generating files

In [ ]:
def automate():
    for steps in range(check_freq, total_timesteps+1, check_freq):
        marlio = f"best_model_{steps}.zip"
        model = PPO.load(f"./train/{marlio}")
        print(f"using {marlio}")
        print()

        action_dict = pred(model)
        filename = "./overlays/" + marlio.split(".")[0] + ".mp4"
        generate_video(combo_images, action_dict, filename, 60)

### Let's automate the boring stuff!

In [ ]:
automate()

In [ ]:
bk2_to_mp4()